In [14]:
# IMPORTS
import cvxpy as cp
import numpy as np
import numpy.linalg
import numpy.matlib
from numpy.linalg import norm
import algorithms.tikhonov_states as tikhonov
from scipy import interpolate
from scipy import signal
import csv   
import matplotlib.pyplot as plt
import pandas as pd

In [15]:
# LOAD DATA

# 1) LOAD DYNAMICS AND TIME ARRAYS

filepath = 'data_files/Duffing_dynamics.csv'
dynamics = np.genfromtxt(filepath, delimiter=',')

t = dynamics[0]
data = dynamics[1:]
n_states = int(data.shape[0] / 2)
X = data[0:n_states]
DX = data[n_states:]
t_samples = len(t)

# noise_lvls = [0.001, 0.01, 0.1, 1, 2]
noise_lvls = [1]

n_noise_lvls = len(noise_lvls)

mean_X_error = np.zeros((n_states, n_noise_lvls))
mean_dX_error = np.zeros((n_states, n_noise_lvls))

var_X_error = np.zeros((n_states, n_noise_lvls))
var_dX_error = np.zeros((n_states, n_noise_lvls))  

for noise, sigma in enumerate(noise_lvls):

    # 2) LOAD OBSERVATION SAMPLES
    Y_samples = []
    noise_lvl = str(sigma)

    for state in range(n_states):

        filepath = 'data_files/Duffing_samples_noise_' + noise_lvl +'_Y'+ str(state+1) + '.csv'
        y = np.genfromtxt(filepath, delimiter=',')

        Y_samples.append(y)

    n_samples = Y_samples[0].shape[0]

    ################### DENOISING AND NUMERICAL DIFFERENTIATION - SMOOTHING SPLINES ###################
    trim = int(np.floor(5 / 100 * t_samples))
    error_X = np.zeros((n_samples,n_states))
    error_dX = np.zeros((n_samples,n_states))
    tt_samples = t_samples - 2 * trim
    Yhat_tik = np.zeros((n_samples, tt_samples))
    DYhat_tik = np.zeros((n_samples, tt_samples))

    for state in range(n_states):

        for sample in range(n_samples):#n_samples
            
            #Find corner point
            y = Y_samples[state][sample,:]
            tik_lc = tikhonov.lcurve_corner(y, lambda_min = 1e2, lambda_max = 10000000000)
            y_tik = tik_lc[0]
            y_tik_ss = interpolate.splrep(t, y_tik, k=3, s=0)
            dy_tik = interpolate.splev(t, y_tik_ss, der=1)
            
            #Remove ends
            Yhat_tik[sample,:] = y_tik[trim:-trim]
            DYhat_tik[sample,:] = dy_tik[trim:-trim]
            
            #Compute errors
            error_X[sample,state] = norm(Yhat_tik[sample,:] - X[state,trim:-trim]) / norm(X[state,trim:-trim])
            error_dX[sample,state] = norm(DYhat_tik[sample,:] - DX[state,trim:-trim]) / norm(DX[state,trim:-trim])
        
        filename_y = 'data_files/tikhonov_hp/Duffing_tikhonov_' + noise_lvl + '_Y' + str(state+1) + '.csv'
        filename_dy = 'data_files/tikhonov_hp/Duffing_tikhonov_' + noise_lvl + '_dY' + str(state+1) + '.csv'
        with open(filename_y, 'w') as csvfile:   
            # creating a csv writer object   
            csvwriter = csv.writer(csvfile)   
            # writing the data rows   
            csvwriter.writerows(Yhat_tik)
        with open(filename_dy, 'w') as csvfile:   
            # creating a csv writer object   
            csvwriter = csv.writer(csvfile)   
            # writing the data rows   
            csvwriter.writerows(DYhat_tik)

        mean_X_error[state, noise] = error_X[:,state].mean()
        mean_dX_error[state, noise] = error_dX[:,state].mean()
        var_X_error[state, noise] = error_X[:,state].var()
        var_dX_error[state, noise] = error_dX[:,state].var()
        
      

  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.


  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.


In [16]:

temp = [['$\sigma$ = ' + str(x)] * n_states for x in noise_lvls]
noise_indx = np.array([item for sublist in temp for item in sublist])

state_indx = np.array(['state x' + str(x+1) for x in range(n_states)] * n_noise_lvls)

arrays = [noise_indx, state_indx]
data = np.vstack((mean_X_error.T.flatten(), var_X_error.T.flatten(), mean_dX_error.T.flatten(), var_dX_error.T.flatten())).T

col_names = ['mean error x', 'var. error x', 'mean error dx', 'var. error dx']
df = pd.DataFrame(data, index=arrays, columns=col_names)
df

mean error x  var. error x  mean error dx  \
$\sigma$ = 1 state x1      0.188412      0.004075       0.317526   
             state x2      0.114062      0.000953       0.312689   

                       var. error dx  
$\sigma$ = 1 state x1       0.010512  
             state x2       0.008616

In [ ]:
#Cheat sheet

#noise = 0.001 -- > lambda_min = 1e-12, lambda_max = 10